In [1]:
import pandas as pd

In [4]:
data_petshop = pd.read_csv('1655023Pet shop.csv')
data_petshop.head()

,Unnamed: 0,products_sold,product_category,quality,satisfaction,discount,retail_price,perc_physical,market_size
0,1,67648,health,off_brand,3.5,12,33.0,42.3,852
1,2,120421,other,off_brand,4.5,10,16.0,54.3,1045
2,3,95242,toys,off_brand,2.9,6,25.0,88.2,1461
3,4,105632,health,off_brand,2.6,8,NaN,5.0,1579
4,5,221627,toys,off_brand,4.9,11,25.0,57.0,2643
